In [ ]:

import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data=pd.read_csv("../input/marrket-mix-dataset/mktmix.csv")

In [ ]:
data.head()

In [ ]:
data.shape


In [ ]:
pd.options.display.float_format = '{:.2f}'.format
data.describe()

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
print(data["Base_Price"].describe())

In [ ]:
data.Base_Price.plot(kind='hist')

In [ ]:
data.boxplot(column='Base_Price')


In [ ]:
data['Base_Price'].quantile(np.arange(0,1,0.1))

In [ ]:
q = data['Base_Price'].quantile(0.01)

In [ ]:
print(q)

In [ ]:
data[data.Base_Price < q]

In [ ]:
avg =data['Base_Price'].mean()

In [ ]:
data.loc[(data["Base_Price"]<q), 'Base_Price'] = avg

In [ ]:
data['Base_Price'].describe()

In [ ]:
data['NewVolSales'].describe()

In [ ]:
data.NewVolSales.plot(kind='hist')

In [ ]:
#new = data.loc[(data["NewVolSales"]>20050.53)]
#nl = list(new.index)
#data.drop(nl,axis = 0)

In [ ]:
data['Radio '].describe()

In [ ]:
R = data['Radio ']

Rd = DataFrame(R)

In [ ]:
type(R)

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(Rd)
X = pd.DataFrame(data = imp.transform(Rd))

In [ ]:

data = pd.concat([data,X],axis = 1)


In [ ]:
data.head()

In [ ]:
data.rename(columns={0:"NRadio"}, inplace=True)
data.head()

In [ ]:
#df.drop('Radio', axis=1, inplace=True)

In [ ]:
#Another way
data['Radio '].isnull().sum()

In [ ]:
# Replace Missing values 
data['Radio']=data['Radio '].fillna(data['Radio '].mean())


In [ ]:
data['Radio'].describe()

In [ ]:
data['Website_Campaign '].value_counts(dropna = False)

In [ ]:
#Visualization

In [ ]:
# Base Price
data.plot(x="NewVolSales",y="Base_Price",kind="scatter")

In [ ]:
#Radio
data.plot(x="NewVolSales",y="Radio",kind="scatter")

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
data.corr()

In [ ]:
import seaborn as sns
corr = data.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [ ]:
new = data.corr()
new.to_csv('corr.csv', sep=',')


In [ ]:
data.rename(columns={'Website_Campaign ':'Website_Campaign'}, inplace=True)


In [ ]:
data.rename(columns={'TV ':"TV"}, inplace=True)

In [ ]:
data['log_sales'] = np.log(data.NewVolSales)


In [ ]:
def d(x):
    if x < 15:
        return "Low"
    elif 15< x < 15.33:
        return "Medium"
    else:
        return "High"
    

In [ ]:
data['BKTPrice']=data['Base_Price'].map(d)

In [ ]:
data.head()

In [ ]:
data['BKTPrice'].value_counts()

In [ ]:
data = pd.get_dummies(data, columns=['NewspaperInserts'])

In [ ]:
data = pd.get_dummies(data, columns=['Website_Campaign'])

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data['Online'] = data['Website_Campaign_Facebook']+ data['Website_Campaign_Twitter']+ data['Website_Campaign_Website Campaign ']

In [ ]:

data["Offline"]=data['TV']+data['InStore']+ data['Radio']
data.columns

In [ ]:

import statsmodels.formula.api as smf

In [ ]:
reg=smf.ols("NewVolSales~Base_Price+InStore+TV+Discount+Stout+Radio+Online+NewspaperInserts_Insert",data=data)

In [ ]:
results=reg.fit()

In [ ]:
print(results.summary())

In [ ]:
reg=smf.ols("NewVolSales~Base_Price+InStore+TV+Discount+Stout",data=data)

In [ ]:
results=reg.fit()

In [ ]:
print(results.summary())

In [ ]:
predictions=results.predict(data)

In [ ]:
actuals=data['NewVolSales']

In [ ]:
## Actual vs Predicted plot
plt.plot(actuals,"b")
plt.plot(predictions,"r")

In [ ]:
residuals = results.resid
type(residuals)

In [ ]:
residualsdf = DataFrame(residuals)
residualsdf.rename(columns={0:"res"}, inplace=True)

In [ ]:

plt.scatter(residualsdf, predictions) 
plt.xticks([])
    

In [ ]:
import sklearn.metrics as metrics

In [ ]:
## Mean Absolute error
mae = metrics.mean_absolute_error(actuals,predictions)
mae

In [ ]:
np.mean(abs((actuals - predictions)/actuals)) #MAPE

In [ ]:
from patsy import dmatrices

In [ ]:
 y, X = dmatrices('NewVolSales~Base_Price+InStore+TV+Discount+Stout', 
                  data=data, return_type='dataframe')

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
## VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [ ]:
vif.round(1)